In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [2]:
PSA_Pokemon_df = pd.read_csv('PWCC_Pokemon.csv')
PSA_Pokemon_df_8 = PSA_Pokemon_df[(PSA_Pokemon_df.Rating == 8)]
PSA_Pokemon_df_10 = PSA_Pokemon_df[(PSA_Pokemon_df.Rating == 10)]

In [3]:
uuid_8 = [[y.split('[')[1].split(']')[0].split(', ')[x][1:-1] for x in range(len(y.split('[')[1].split(']')[0].split(', ')))] for y in list(PSA_Pokemon_df_8.uuid)]
uuid_10 = [[y.split('[')[1].split(']')[0].split(', ')[x][1:-1] for x in range(len(y.split('[')[1].split(']')[0].split(', ')))] for y in list(PSA_Pokemon_df_10.uuid)]

In [4]:
available_eights = [f.split('.jpg')[0] for f in listdir('PWCC/8') if isfile(join('PWCC/8', f))]
available_tens = [f.split('.jpg')[0] for f in listdir('PWCC/10') if isfile(join('PWCC/10', f))]

eights_front = []
eights_back = []
for i in uuid_8:
    if (i[0] in available_eights) and (i[0] in available_eights):
        eights_front.append('PWCC/8/'+str(i[0])+'.jpg')
        eights_back.append('PWCC/8/'+str(i[1])+'.jpg')
        
tens_front = []
tens_back = []
for i in uuid_10:
    if (i[0] in available_tens) and (i[0] in available_tens):
        tens_front.append('PWCC/10/'+str(i[0])+'.jpg')
        tens_back.append('PWCC/10/'+str(i[1])+'.jpg')

In [5]:
sample_size = 3000

image_front = eights_front[:sample_size] + tens_front[:sample_size]
image_back = eights_back[:sample_size] + tens_back[:sample_size]

y = [1 for i in tens_front[:sample_size]] + [0 for i in eights_front[:sample_size]]
y = pd.DataFrame(y)
y.value_counts()

0    3000
1    3000
dtype: int64

In [6]:
from tqdm.notebook import tqdm
from PIL import Image

res = 224

def load_image(file):
    try:
        image = Image.open(
            file
        ).convert('LA').resize((res, res))
        arr = np.array(image)
    except:
        print('didnt work')
        arr = np.zeros((res, res, 2))
    return arr

x_front = np.array([load_image(i) for i in tqdm(image_front)])
x_back = np.array([load_image(i) for i in tqdm(image_back)])

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

In [7]:
from sklearn.model_selection import train_test_split

x_front_tr, x_front_vl, x_back_tr, x_back_vl, y_tr, y_vl = train_test_split(
    x_front,
    x_back,
    y,
    test_size=0.2)

In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, GRU, LSTM, Bidirectional, TimeDistributed, Attention, Dropout
from tensorflow.keras.optimizers import Adam, SGD

in_front = keras.Input(batch_shape=(None, (res), (res), 2))
in_back = keras.Input(batch_shape=(None, (res), (res), 2))

#Front
cov_0 = Conv2D(32, (5, 5), activation='relu')(in_front)
pool_0 = MaxPool2D((2, 2))(cov_0)
cov_1 = Conv2D(64, (5, 5), activation='relu')(pool_0)
pool_1 = MaxPool2D((2, 2))(cov_1)
cov_2 = Conv2D(64, (5, 5), activation='relu')(pool_1)
pool_2 = MaxPool2D((2, 2))(cov_2)
cov_3 = Conv2D(64, (5, 5), activation='relu')(pool_2)
pool_3 = MaxPool2D((6, 6))(cov_3)
flatten = Flatten()(pool_3)

#Back
cov_0_1 = Conv2D(32, (5, 5), activation='relu')(in_back)
pool_0_1 = MaxPool2D((2, 2))(cov_0_1)
cov_1_1 = Conv2D(64, (5, 5), activation='relu')(pool_0_1)
pool_1_1 = MaxPool2D((2, 2))(cov_1_1)
cov_2_1 = Conv2D(64, (5, 5), activation='relu')(pool_1_1)
pool_2_1 = MaxPool2D((2, 2))(cov_2_1)
cov_3_1 = Conv2D(64, (5, 5), activation='relu')(pool_2_1)
pool_3_1 = MaxPool2D((6, 6))(cov_3_1)
flatten_1 = Flatten()(pool_3_1)


fused = tf.concat([flatten, flatten_1], axis=-1)

fc = Dense(64, activation='relu')(fused)
pred = Dense(1, activation='sigmoid')(fc)

model = keras.Model(
    inputs={
        'in_front': in_front,
        'in_back': in_back
    },
    outputs=pred,
)

In [9]:
model.compile(
    optimizer=Adam(lr=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 2) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 2) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 220, 220, 32) 1632        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 220, 220, 32) 1632        input_2[0][0]                    
______________________________________________________________________________________________

In [10]:
history = model.fit(
    x={
        'in_front': x_front_tr,
        'in_back': x_back_tr
    },
    y=y_tr,
    
    epochs=100,
    batch_size=32,

    validation_data=(
        {
            'in_front': x_front_vl,
            'in_back': x_back_vl
        },
        y_vl
    ),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ],
    
    verbose=1

)

Epoch 1/100
150/150 [==============================] - 324s 2s/step - loss: 2.9561 - accuracy: 0.5775 - auc: 0.5925 - val_loss: 0.6314 - val_accuracy: 0.6258 - val_auc: 0.7988
Epoch 2/100
150/150 [==============================] - 322s 2s/step - loss: 0.5656 - accuracy: 0.7080 - auc: 0.7815 - val_loss: 0.5175 - val_accuracy: 0.7367 - val_auc: 0.8238
Epoch 3/100
150/150 [==============================] - 326s 2s/step - loss: 0.5258 - accuracy: 0.7303 - auc: 0.8132 - val_loss: 0.5285 - val_accuracy: 0.7233 - val_auc: 0.8264
Epoch 4/100
150/150 [==============================] - 322s 2s/step - loss: 0.4729 - accuracy: 0.7693 - auc: 0.8582 - val_loss: 0.6359 - val_accuracy: 0.6867 - val_auc: 0.8367
Epoch 5/100
150/150 [==============================] - 328s 2s/step - loss: 0.4602 - accuracy: 0.7765 - auc: 0.8631 - val_loss: 0.5246 - val_accuracy: 0.7342 - val_auc: 0.8511
Epoch 6/100
150/150 [==============================] - 324s 2s/step - loss: 0.4199 - accuracy: 0.8046 - auc: 0.8887 - va